In [1]:
import boto3 
import batch_functions as bf 
import configparser 
import numpy as np 

In [2]:
KEY            ='G:\\' 				# Where the files to upload are stored 
BUCKET         ='crary'				# Bucket destination (should not change)
RESULTPATH     ='BronxVA'			# Destination folder for files in s3 
MAX_FILES      = 20000 				# Max number of files to upload 
TEST           = False  			# Set to false to use/true to test 

In [3]:
config = configparser.ConfigParser() 
config.sections() 

config.read('s3.config')

YOUR_ACCESS_KEY=config['CREDENTIALS']['YOUR_ACCESS_KEY'] 
YOUR_SECRET_KEY=config['CREDENTIALS']['YOUR_SECRET_KEY']

In [4]:
# Create a connection to S3 
s3 = boto3.client(
	's3', 
	aws_access_key_id=YOUR_ACCESS_KEY,
	aws_secret_access_key=YOUR_SECRET_KEY
)

# query s3 for files already in RESULTPATH 
def getFiles(s3Client, bucket=BUCKET, prefix=RESULTPATH): 
	kwargs = {'Bucket': bucket, 'Prefix': prefix}

	iterate = True
	i = 0 
	files = {}
	while iterate: 
		resp = s3Client.list_objects_v2(**kwargs)
		for obj in resp['Contents']:
			filename = obj['Key']
			files[i] = filename 
			i+=1 

		try: 
			kwargs['ContinuationToken'] = resp['NextContinuationToken']
		except KeyError: 
			iterate = False 
	return files 

def getSaveFileLocation(): return "bucket=" + BUCKET + "_" + RESULTPATH + "_contents.csv"

In [12]:
filesInS3 = np.array(list(getFiles(s3).values()))
filesInS3 = np.array([f.split('BronxVA/')[1].replace('\\', '/') for f in filesInS3])
filesInS3.sort()

In [16]:
filesOnDrive = bf.getAll('G:\\')
filesOnDrive = [f.split('G:\\')[1].replace('\\', '/') for f in filesOnDrive]
filesOnDrive.sort()

In [19]:
print(filesInS3[1500])
print(filesOnDrive[1500-1])
print(filesInS3[1500] in filesOnDrive)

XE16-010_1_HE_1/Data0014.dat
XE16-010_1_HE_1/Data0014.dat
True


In [20]:
print(len(filesInS3))
print(len(filesOnDrive))

16704
16703


In [21]:
missingfiles = []
for fn in filesInS3: 
    if fn not in filesOnDrive: 
        missingfiles = np.append(np.array(missingfiles), np.array(fn))
        
print(missingfiles)

['']


In [22]:
unique_elements, counts_elements = np.unique(filesInS3, return_counts=True)

In [23]:
for i in range(0, len(unique_elements)): 
    if counts_elements[i] > 1: print(unique_elements[i], counts_elements[i])